In [1]:
from model.MF import *
from preprocess.AmazonBook import *
from evaluation.MF_SISA import *
pd.options.display.max_rows = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path = './dataset/amazon-book'
dataset = AmazonBook(path)

data = dataset.get()
num_users, num_books = dataset.getNumber()
config = {
    'k': 20,
    'lr': 1e-4, 
    'epochs': 100,
    'num_layers': 2,
    'batch_size': 8192,
    'embedding_dim': 64,
    'num_users': num_users,
    'num_books': num_books,
    'tuning_type': None,
    "weight_decay": 1e-7,
    'global_bias':(data.edge_index.size(1) + data.edge_label_index.size(1) + 2) / (num_books * num_users)
}
model = MF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    mean = config['global_bias'],
    embedding_dim = config['embedding_dim']
).to(device)

# SISA Method

In [2]:
config['epochs'] = 60  # enough to converge
config['num_shards'] = 5
model_list =[]

for _ in range(config['num_shards']):
    model_list.append(MF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    mean = config['global_bias'],
    embedding_dim = config['embedding_dim']
).to(device))

shard_models, shards, epoch_tracks, test_topks = sisa_MF_eva(model_list, config, data, device)

Epoch: 001, Loss: 0.9945, HR@20: 0.0001, Recall@20: 0.0003, NDCG@20: 0.0002
Epoch: 002, Loss: 0.9844, HR@20: 0.0001, Recall@20: 0.0003, NDCG@20: 0.0002
Epoch: 003, Loss: 0.9743, HR@20: 0.0002, Recall@20: 0.0003, NDCG@20: 0.0003
Epoch: 004, Loss: 0.9642, HR@20: 0.0002, Recall@20: 0.0004, NDCG@20: 0.0003
Epoch: 005, Loss: 0.9541, HR@20: 0.0003, Recall@20: 0.0006, NDCG@20: 0.0005
Epoch: 006, Loss: 0.9438, HR@20: 0.0005, Recall@20: 0.0009, NDCG@20: 0.0008
Epoch: 007, Loss: 0.9333, HR@20: 0.0007, Recall@20: 0.0014, NDCG@20: 0.0013
Epoch: 008, Loss: 0.9226, HR@20: 0.0011, Recall@20: 0.0020, NDCG@20: 0.0018
Epoch: 009, Loss: 0.9116, HR@20: 0.0015, Recall@20: 0.0027, NDCG@20: 0.0024
Epoch: 010, Loss: 0.9001, HR@20: 0.0018, Recall@20: 0.0034, NDCG@20: 0.0030
Epoch: 011, Loss: 0.8882, HR@20: 0.0022, Recall@20: 0.0040, NDCG@20: 0.0036
Epoch: 012, Loss: 0.8758, HR@20: 0.0025, Recall@20: 0.0047, NDCG@20: 0.0041
Epoch: 013, Loss: 0.8629, HR@20: 0.0027, Recall@20: 0.0051, NDCG@20: 0.0044
Epoch: 014, 

# Split the dataset

In [6]:
# Basic setting
num_edges = data.edge_index.size(1)
perm = torch.randperm(num_edges)
split = int(num_edges * 0.1)  # 10% of edges will be retained

# Define the data for forget and retain dataset
forget_data = Data()
retain_data = Data()

forget_data.num_nodes = data.num_nodes
retain_data.num_nodes = data.num_nodes
forget_data.edge_index = data.edge_index[:, perm[:split]]
retain_data.edge_index = data.edge_index[:, perm[split:]]
forget_data.edge_label_index = data.edge_label_index
retain_data.edge_label_index = data.edge_label_index

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
shard_models = sisa_MF_unlearning_eva(shard_models, shards, retain_data, forget_data, config, device)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [5]:
shard_models = sisa_MF_unlearning_eva(shard_models, shards, retain_data, forget_data, config, device)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
shard_models = sisa_MF_unlearning_eva(shard_models, shards, retain_data, forget_data, config, device)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
shard_models = sisa_MF_unlearning_eva(shard_models, shards, retain_data, forget_data, config, device)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
shard_models = sisa_MF_unlearning_eva(shard_models, shards, retain_data, forget_data, config, device)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
sisa_MF_forget_data_eva(shard_models, forget_data, config, device)